In [3]:
import numpy as np
import pandas as pd
from pandasql import sqldf 
from ast import literal_eval
import helper_functions as hf

## Reading files

In [4]:
# read in files needed for feature engineering
regular_season_compact_all = pd.read_csv('data/kaggle_data/MRegularSeasonCompactResults.csv')
regular_season_detailed = pd.read_csv('data/kaggle_data/MRegularSeasonDetailedResults.csv')
tourney_compact_all = pd.read_csv('data/kaggle_data/MNCAATourneyCompactResults.csv')
tourney_detailed = pd.read_csv('data/kaggle_data/MNCAATourneyDetailedResults.csv')
teams = pd.read_csv('data/kaggle_data/MTeams.csv')
seeds_all = pd.read_csv('data/kaggle_data/MNCAATourneySeeds.csv')
slots_all = pd.read_csv('data/kaggle_data/MNCAATourneySlots.csv')
ordinals = pd.read_csv('data/kaggle_data/MMasseyOrdinals_thruDay128.csv')

In [5]:
# trim everything to start in 2003
min_year = 2003
regular_season_compact = regular_season_compact_all[regular_season_compact_all['Season'] >= min_year].reset_index(drop=True)
tourney_compact = tourney_compact_all[tourney_compact_all['Season'] >= min_year].reset_index(drop=True)
seeds = seeds_all[seeds_all['Season'] >= min_year].reset_index(drop=True)
slots = slots_all[slots_all['Season'] >= min_year].reset_index(drop=True)

## Look at data

In [6]:
regular_season_compact.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,2003,10,1104,68,1328,62,N,0
1,2003,10,1272,70,1393,63,N,0
2,2003,11,1266,73,1437,61,N,0
3,2003,11,1296,56,1457,50,N,0
4,2003,11,1400,77,1208,71,N,0


In [7]:
regular_season_detailed.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [8]:
tourney_compact.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,2003,134,1421,92,1411,84,N,1
1,2003,136,1112,80,1436,51,N,0
2,2003,136,1113,84,1272,71,N,0
3,2003,136,1141,79,1166,73,N,0
4,2003,136,1143,76,1301,74,N,1


In [9]:
tourney_detailed.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [10]:
teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2022
1,1102,Air Force,1985,2022
2,1103,Akron,1985,2022
3,1104,Alabama,1985,2022
4,1105,Alabama A&M,2000,2022


In [11]:
seeds.head()

,Season,Seed,TeamID
0,2003,W01,1328
1,2003,W02,1448
2,2003,W03,1393
3,2003,W04,1257
4,2003,W05,1280


In [34]:
seeds[seeds['Season'] == 2022]

,Season,Seed,TeamID
1200,2022,W01,1124
1201,2022,W02,1246
1202,2022,W03,1345
1203,2022,W04,1417
1204,2022,W05,1388
...,...,...,...
1263,2022,Z13,1151
1264,2022,Z14,1255
1265,2022,Z15,1174
1266,2022,Z16a,1136


In [39]:
slots.head()

,Season,Slot,StrongSeed,WeakSeed
0,2003,R1W1,W01,W16
1,2003,R1W2,W02,W15
2,2003,R1W3,W03,W14
3,2003,R1W4,W04,W13
4,2003,R1W5,W05,W12


## Feature engineering

### Adding derived game stats to regular_season_detailed

Create:
- possessions

Which will then be used to calculate:
- pace
- offensive efficiency
- defensive efficiency

Basic Possession Formula = 0.96*[(Field Goal Attempts)+(Turnovers)+0.44*(Free Throw Attempts)-(Offensive Rebounds)]

Pace Calculation Formula = [200/(Team Minutes)]*(Possession_team+Possession_opponent)/2

Offensive Efficiency Formula = 100*(Points Scored / Possessions)

Defensive Efficiency Formula = 100*(Opp Points Scored / Opp Possessions) = Opponent Offensive Efficiency

In [40]:
# winning team possessions
regular_season_detailed['WPos'] = \
    0.96 * (regular_season_detailed['WFGA'] + regular_season_detailed['WTO'] + \
     0.44 * regular_season_detailed['WFTA'] - regular_season_detailed['WOR'])

# losing team possessions
regular_season_detailed['LPos'] = \
    0.96 * (regular_season_detailed['LFGA'] + regular_season_detailed['LTO'] + \
     0.44 * regular_season_detailed['LFTA'] - regular_season_detailed['LOR'])

# pace
regular_season_detailed['Pace'] =  \
    (200/(200 + 5 * 5 * regular_season_detailed['NumOT'])) * \
    (regular_season_detailed['WPos'] + regular_season_detailed['LPos']) / 2

# winning team offensive efficiency, losing team defensive efficiency
regular_season_detailed['WOffEff'] = \
    100*(regular_season_detailed['WScore'] / regular_season_detailed['WPos'])

# losing team offensive efficiency, winning team defensive efficiency
regular_season_detailed['LOffEff'] = \
    100*(regular_season_detailed['LScore'] / regular_season_detailed['LPos'])

### General season and per-game team stats

Create: 
- games played
- regular season wins
- reg. season losses
- avg. game margin
- std. dev. game margin
- win %
- away/neutral wins
- home losses
- close wins
- close losses
- avg. points for
- avg. points against
- total 3 pointers made 
- total 3 pointers attempted
- 3 pt. %
- total FT made
- total FT attempted
- FT %
- avg. TOVs
- avg. offensive efficiency
- avg. defensive efficiency
- avg. pace

In [41]:
regular_season_detailed.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LAst,LTO,LStl,LBlk,LPF,WPos,LPos,Pace,WOffEff,LOffEff
0,2003,10,1104,68,1328,62,N,0,27,58,...,8,18,9,2,20,71.9232,67.8528,69.8880,94.545293,91.374269
1,2003,10,1272,70,1393,63,N,0,26,62,...,7,12,8,6,16,65.6256,65.0880,65.3568,106.665691,96.792035
2,2003,11,1266,73,1437,61,N,0,24,58,...,9,12,2,5,23,61.2096,61.5552,61.3824,119.262338,99.098045
3,2003,11,1296,56,1457,50,N,0,18,38,...,9,19,4,3,23,55.3344,55.2960,55.3152,101.202868,90.422454
4,2003,11,1400,77,1208,71,N,0,30,61,...,12,10,7,1,14,61.1712,60.3648,60.7680,125.876229,117.618215


In [43]:
regular_season_detailed.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
       'WPos', 'LPos', 'Pace', 'WOffEff', 'LOffEff'],
      dtype='object')

In [44]:
# aggregate data for when team is the winning team
wins_q = '''
SELECT 
    Season, WTeamID AS TeamID, 
    COUNT() as num_wins,
    GROUP_CONCAT(WScore - LScore) as winning_margins,
    SUM(WScore) sum_score,
    SUM(LScore) sum_score_of_opponents,
    SUM(CASE WHEN WLoc == 'A' OR WLoc == 'N' THEN 1 ELSE 0 END) non_home_wins,
    SUM(CASE WHEN WScore - LScore <= 3 OR NumOT > 0 THEN 1 ELSE 0 END) close_wins,
    SUM(WFGA3) sum_3FGA,
    SUM(WFGM3) sum_3FGM,
    SUM(WFTA) sum_FTA,
    SUM(WFTM) sum_FTM,
    SUM(WTO) sum_TOV,
    SUM(Pace) sum_pace,
    SUM(WOffEff) sum_off_eff,
    SUM(LOffEff) sum_def_eff
FROM regular_season_detailed
GROUP BY 1, 2
'''
wins = sqldf(wins_q)
wins.head()

,Season,TeamID,num_wins,winning_margins,sum_score,sum_score_of_opponents,non_home_wins,close_wins,sum_3FGA,sum_3FGM,sum_FTA,sum_FTM,sum_TOV,sum_pace,sum_off_eff,sum_def_eff
0,2003,1102,12,"29,5,21,14,9,18,38,13,23,4,5,8",825,638,3,0,259,120,249,163,133,650.937600,1530.926442,1170.195361
1,2003,1103,13,"4,7,33,5,1,18,1,16,11,2,1,15,8",1141,1019,4,4,187,71,402,290,163,873.817173,1649.785724,1459.595972
2,2003,1104,17,"6,26,15,6,28,9,9,8,18,7,10,6,9,16,5,23,23",1270,1046,4,1,354,120,383,272,222,1104.100267,1953.578718,1596.765971
3,2003,1105,7,"19,7,4,1,18,34,8",556,465,2,1,157,64,180,134,126,515.424000,755.667122,631.810684
4,2003,1106,13,"5,5,12,9,10,6,1,18,26,24,11,4,4",888,753,5,2,207,76,270,168,230,844.194133,1362.130906,1140.286408


In [45]:
# aggregate data for when team is the losing team
losses_q = '''
SELECT 
    Season, LTeamID AS TeamID, 
    COUNT() as num_losses,
    GROUP_CONCAT(LScore - WScore) as losing_margins,
    SUM(LScore) sum_score,
    SUM(WScore) sum_score_of_opponents,
    SUM(CASE WHEN WLoc == 'A' THEN 1 ELSE 0 END) home_losses,
    SUM(CASE WHEN WScore - LScore <= 3 OR NumOT > 0 THEN 1 ELSE 0 END) close_losses,
    SUM(LFGA3) sum_3FGA,
    SUM(LFGM3) sum_3FGM,
    SUM(LFTA) sum_FTA,
    SUM(LFTM) sum_FTM,
    SUM(LTO) sum_TOV,
    SUM(Pace) sum_pace,
    SUM(LOffEff) sum_off_eff,
    SUM(WOffEff) sum_def_eff
FROM regular_season_detailed
GROUP BY 1, 2
'''
losses = sqldf(losses_q)
losses.head()

,Season,TeamID,num_losses,losing_margins,sum_score,sum_score_of_opponents,home_losses,close_losses,sum_3FGA,sum_3FGM,sum_FTA,sum_FTM,sum_TOV,sum_pace,sum_off_eff,sum_def_eff
0,2003,1102,16,"-18,-13,-17,-10,-15,-2,-2,-10,-32,-14,-7,-14,-...",778,958,4,2,324,99,230,149,187,811.584000,1528.733279,1896.446461
1,2003,1103,14,"-3,-3,-2,-6,-8,-14,-15,-7,-16,-7,-16,-2,-4,-2",986,1091,5,6,247,76,296,224,178,878.963200,1499.794451,1685.607938
2,2003,1104,11,"-2,-1,-9,-19,-17,-11,-19,-5,-4,-4,-13",670,774,2,2,202,58,203,144,150,664.608000,1095.144346,1290.023656
3,2003,1105,19,"-28,-42,-13,-2,-22,-14,-10,-1,-1,-6,-5,-12,-31...",1310,1528,7,7,383,133,388,267,359,1342.574933,1788.600451,2134.194163
4,2003,1106,15,"-1,-12,-26,-16,-1,-9,-20,-6,-6,-1,-3,-19,-6,-1...",893,1032,4,5,287,95,191,130,247,949.804800,1393.217658,1646.962777


In [46]:
# outer join wins and losses to get all teams W/L record each season
team_season_stats = wins.merge(losses, on=['Season', 'TeamID'], how='outer', suffixes=('_w', '_l'))
team_season_stats.fillna(0, inplace=True)

# typecast columns
team_season_stats[['num_wins','num_losses']] = team_season_stats[['num_wins','num_losses']].astype(int)
team_season_stats[['winning_margins','losing_margins']] = team_season_stats[['winning_margins','losing_margins']].astype(str)

# total games
team_season_stats['total_games'] = team_season_stats['num_wins'] + team_season_stats['num_losses']

# total points for
team_season_stats['total_points_for'] = team_season_stats['sum_score_w'] + team_season_stats['sum_score_l']

# total points against
team_season_stats['total_points_against'] = team_season_stats['sum_score_of_opponents_w'] + team_season_stats['sum_score_of_opponents_l']

# win pct.
team_season_stats['win_pct'] = team_season_stats['num_wins'] / team_season_stats['total_games']

# avg. points for
team_season_stats['avg_points_for'] = team_season_stats['total_points_for'] / team_season_stats['total_games']

# avg. points against
team_season_stats['avg_points_against'] = team_season_stats['total_points_against'] / team_season_stats['total_games']

# 3FGA per game
team_season_stats['3FGA_pg'] = (team_season_stats['sum_3FGA_w'] + team_season_stats['sum_3FGA_l']) / team_season_stats['total_games']

# 3FGM per game
team_season_stats['3FGM_pg'] = (team_season_stats['sum_3FGM_w'] + team_season_stats['sum_3FGM_l']) / team_season_stats['total_games']

# 3 pt. pct.
team_season_stats['3_pct'] = team_season_stats['3FGM_pg'] / team_season_stats['3FGA_pg']

# FTA per game
team_season_stats['FTA_pg'] = (team_season_stats['sum_FTA_w'] + team_season_stats['sum_FTA_l']) / team_season_stats['total_games']

# FTM per game
team_season_stats['FTM_pg'] = (team_season_stats['sum_FTM_w'] + team_season_stats['sum_FTM_l']) / team_season_stats['total_games']

# FT pct.
team_season_stats['FT_pct'] = team_season_stats['FTM_pg'] / team_season_stats['FTA_pg']

# TOV per game
team_season_stats['TOV_pg'] = (team_season_stats['sum_TOV_w'] + team_season_stats['sum_TOV_l']) / team_season_stats['total_games']

# turn strings of winning and losing margins into one tuple
team_season_stats['differential_arr'] = [literal_eval(wm + ',' + lm) for wm, lm in zip(team_season_stats['winning_margins'], team_season_stats['losing_margins'])]

# avg. score differential
team_season_stats['avg_game_margin'] = [np.mean(arr) for arr in team_season_stats['differential_arr']]

# std. dev. of score differential
team_season_stats['std_game_margin'] = [np.std(arr) for arr in team_season_stats['differential_arr']]

# avg. pace
team_season_stats['avg_pace'] = (team_season_stats['sum_pace_w'] + team_season_stats['sum_pace_l']) / team_season_stats['total_games']

# avg. off. efficiency
team_season_stats['off_eff'] = (team_season_stats['sum_off_eff_w'] + team_season_stats['sum_off_eff_l']) / team_season_stats['total_games']

# avg. def. efficiency
team_season_stats['def_eff'] = (team_season_stats['sum_def_eff_w'] + team_season_stats['sum_def_eff_l']) / team_season_stats['total_games']

# pythagorean win pct. (Morey version: k=13.91)
k = 13.91
team_season_stats['pythag_win_pct'] = (team_season_stats['total_points_for'] ** k) / \
    (team_season_stats['total_points_for'] ** k + team_season_stats['total_points_against'] ** k)

# luck
team_season_stats['luck'] = team_season_stats['win_pct'] - team_season_stats['pythag_win_pct']

In [47]:
team_season_stats_compact = team_season_stats[['Season', 'TeamID', 'total_games', 
                     'num_wins', 'num_losses', 'win_pct',
                     'avg_points_for', 'avg_points_against',
                     'avg_game_margin', 'std_game_margin',
                     'non_home_wins', 'home_losses',
                     'close_wins', 'close_losses',
                     '3FGA_pg', '3FGM_pg', '3_pct',
                     'FTM_pg', 'FTA_pg', 'FT_pct', 'TOV_pg',
                     'avg_pace', 'off_eff', 'def_eff',
                     'pythag_win_pct', 'luck']]
team_season_stats_compact.head()

,Season,TeamID,total_games,num_wins,num_losses,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,...,3_pct,FTM_pg,FTA_pg,FT_pct,TOV_pg,avg_pace,off_eff,def_eff,pythag_win_pct,luck
0,2003,1102,28,12,16,0.428571,57.250000,57.000000,0.250000,15.878163,...,0.375643,11.142857,17.107143,0.651357,11.428571,52.232914,109.273561,109.522922,0.515214,-0.086643
1,2003,1103,27,13,14,0.481481,78.777778,78.148148,0.629630,11.126039,...,0.338710,19.037037,25.851852,0.736390,12.629630,64.917792,116.651118,116.489034,0.527877,-0.046395
2,2003,1104,28,17,11,0.607143,69.285714,65.000000,4.285714,13.149843,...,0.320144,14.857143,20.928571,0.709898,13.285714,63.168152,108.882967,103.099630,0.708513,-0.101370
3,2003,1105,26,7,19,0.269231,71.769231,76.653846,-4.884615,15.552212,...,0.364815,15.423077,21.846154,0.705986,18.653846,71.461497,97.856445,106.384802,0.285796,-0.016565
4,2003,1106,28,13,15,0.464286,63.607143,63.750000,-0.142857,12.374266,...,0.346154,10.642857,16.464286,0.646421,17.035714,64.071390,98.405306,99.544614,0.492199,-0.027913


### Results vs. tourney and non-tourney teams

Create:
- seed number
- seed
- games vs. tourney teams
- wins vs. tourney teams (good wins)
- losses vs. tourney teams (good losses)
- away wins vs. tourney teams (tough wins)
- losses vs. non-tourney teams (bad losses)

In [48]:
# join game results with seeds
team_seeds_q = '''
SELECT 
    a.*, b.Seed WTeamSeed, c.Seed LTeamSeed
FROM regular_season_compact a
LEFT JOIN seeds b
    ON a.WTeamID = b.TeamID
    AND a.Season = b.Season
LEFT JOIN seeds c
    ON a.LTeamID = c.TeamID
    AND a.Season = c.Season
'''
team_seeds = sqldf(team_seeds_q)
# make sure both columns are strings
team_seeds[['WTeamSeed','LTeamSeed']] = team_seeds[['WTeamSeed','LTeamSeed']].astype(str)
team_seeds.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamSeed,LTeamSeed
0,2003,10,1104,68,1328,62,N,0,Y10,W01
1,2003,10,1272,70,1393,63,N,0,Z07,W03
2,2003,11,1266,73,1437,61,N,0,Y03,None
3,2003,11,1296,56,1457,50,N,0,None,None
4,2003,11,1400,77,1208,71,N,0,X01,None


In [49]:
team_seeds[team_seeds['Season'] == 2022]

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamSeed,LTeamSeed
96687,2022,8,1104,93,1256,64,H,0,X06,None
96688,2022,8,1105,82,1398,73,H,0,None,None
96689,2022,8,1110,77,1265,73,H,1,None,None
96690,2022,8,1112,81,1319,52,H,0,Z01,None
96691,2022,8,1113,76,1339,60,H,0,None,None
...,...,...,...,...,...,...,...,...,...,...
102027,2022,132,1222,71,1272,53,N,0,Z05,X09
102028,2022,132,1234,75,1345,66,N,0,Y05,W03
102029,2022,132,1350,64,1172,62,N,0,Y12,X10
102030,2022,132,1397,65,1401,50,N,0,Z03,None


In [54]:
wins_tourney_q = '''
SELECT
    Season, WTeamID AS TeamID, WTeamSeed AS Seed,
    SUM(CASE WHEN LTeamSeed != 'None' THEN 1 ELSE 0 END) wins_vs_tourney_teams,
    SUM(CASE WHEN LTeamSeed != 'None' AND WLoc != 'H' THEN 1 ELSE 0 END) away_wins_vs_tourney_teams
FROM team_seeds
GROUP BY 1, 2
'''
wins_tourney = sqldf(wins_tourney_q)
wins_tourney.head()

,Season,TeamID,Seed,wins_vs_tourney_teams,away_wins_vs_tourney_teams
0,2003,1102,None,1,0
1,2003,1103,None,1,1
2,2003,1104,Y10,5,1
3,2003,1105,None,1,0
4,2003,1106,None,1,0


In [55]:
wins_tourney[wins_tourney['Season'] == 2022]

,Season,TeamID,Seed,wins_vs_tourney_teams,away_wins_vs_tourney_teams
6528,2022,1101,None,1,0
6529,2022,1102,None,1,0
6530,2022,1103,W13,1,1
6531,2022,1104,X06,9,2
6532,2022,1105,None,0,0
...,...,...,...,...,...
6879,2022,1468,None,0,0
6880,2022,1469,None,0,0
6881,2022,1470,None,0,0
6882,2022,1471,None,1,0


In [57]:
losses_tourney_q = '''
SELECT
    Season, LTeamID AS TeamID, LTeamSeed AS Seed,
    SUM(CASE WHEN WTeamSeed != 'None' THEN 1 ELSE 0 END) losses_vs_tourney_teams,
    SUM(CASE WHEN WTeamSeed = 'None' THEN 1 ELSE 0 END) losses_vs_non_tourney_teams
FROM team_seeds
GROUP BY 1, 2
'''
losses_tourney = sqldf(losses_tourney_q)
losses_tourney.head()

,Season,TeamID,Seed,losses_vs_tourney_teams,losses_vs_non_tourney_teams
0,2003,1102,None,7,9
1,2003,1103,None,1,13
2,2003,1104,Y10,6,5
3,2003,1105,None,3,16
4,2003,1106,None,4,11


In [64]:
team_season_stats_tourney = wins_tourney.merge(losses_tourney, on=['Season', 'TeamID'], how='outer', suffixes=('_w', '_l'))
team_season_stats_tourney

,Season,TeamID,Seed_w,wins_vs_tourney_teams,away_wins_vs_tourney_teams,Seed_l,losses_vs_tourney_teams,losses_vs_non_tourney_teams
0,2003,1102,None,1.0,0.0,None,7.0,9.0
1,2003,1103,None,1.0,1.0,None,1.0,13.0
2,2003,1104,Y10,5.0,1.0,Y10,6.0,5.0
3,2003,1105,None,1.0,0.0,None,3.0,16.0
4,2003,1106,None,1.0,0.0,None,4.0,11.0
...,...,...,...,...,...,...,...,...
6887,2015,1212,NaN,NaN,NaN,None,4.0,23.0
6888,2015,1363,NaN,NaN,NaN,None,5.0,23.0
6889,2021,1152,NaN,NaN,NaN,None,4.0,5.0
6890,2022,1175,NaN,NaN,NaN,None,5.0,20.0


In [65]:
team_season_stats_vs_tourney_teams_q = '''
SELECT
    Season, TeamID,
    COALESCE(Seed_w, Seed_l) Seed,
    COALESCE(wins_vs_tourney_teams, 0) wins_vs_tourney_teams,
    COALESCE(away_wins_vs_tourney_teams, 0) away_wins_vs_tourney_teams,
    COALESCE(losses_vs_tourney_teams, 0) losses_vs_tourney_teams,
    COALESCE(losses_vs_non_tourney_teams, 0) losses_vs_non_tourney_teams
FROM team_season_stats_tourney
GROUP BY 1, 2
'''
team_season_stats_vs_tourney_teams = sqldf(team_season_stats_vs_tourney_teams_q)
team_season_stats_vs_tourney_teams.head()

,Season,TeamID,Seed,wins_vs_tourney_teams,away_wins_vs_tourney_teams,losses_vs_tourney_teams,losses_vs_non_tourney_teams
0,2003,1102,None,1.0,0.0,7.0,9.0
1,2003,1103,None,1.0,1.0,1.0,13.0
2,2003,1104,Y10,5.0,1.0,6.0,5.0
3,2003,1105,None,1.0,0.0,3.0,16.0
4,2003,1106,None,1.0,0.0,4.0,11.0


In [66]:
team_season_stats_vs_tourney_teams['games_vs_tourney_teams'] = \
    team_season_stats_vs_tourney_teams['wins_vs_tourney_teams'] + team_season_stats_vs_tourney_teams['losses_vs_tourney_teams']

In [67]:
team_season_stats_vs_tourney_teams.head()

,Season,TeamID,Seed,wins_vs_tourney_teams,away_wins_vs_tourney_teams,losses_vs_tourney_teams,losses_vs_non_tourney_teams,games_vs_tourney_teams
0,2003,1102,None,1.0,0.0,7.0,9.0,8.0
1,2003,1103,None,1.0,1.0,1.0,13.0,2.0
2,2003,1104,Y10,5.0,1.0,6.0,5.0,11.0
3,2003,1105,None,1.0,0.0,3.0,16.0,4.0
4,2003,1106,None,1.0,0.0,4.0,11.0,5.0


### Advanced features

Create:
- RPI/NET ranking (NCAA stopped using RPI in 2018 in favor of NET)
- KENPOM ranking
- momentum (change in RPI/NET and KENPOM)
- wins in previous x tourneys
- wins in previous x seasons

In [68]:
ordinals.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


In [69]:
np.unique(ordinals['SystemName'])

array(['7OT', 'ACU', 'ADE', 'AP', 'ARG', 'ATP', 'AUS', 'AWS', 'BAR',
       'BBT', 'BCM', 'BD', 'BIH', 'BKM', 'BLS', 'BNM', 'BNT', 'BNZ',
       'BOB', 'BOW', 'BP5', 'BPI', 'BRZ', 'BUR', 'BWE', 'CBR', 'CJB',
       'CMV', 'CNG', 'COL', 'COX', 'CPA', 'CPR', 'CRO', 'CRW', 'CTL',
       'CWL', 'D1A', 'DAV', 'DC', 'DC2', 'DCI', 'DDB', 'DES', 'DII',
       'DOK', 'DOL', 'DP', 'DUN', 'DWH', 'EBB', 'EBP', 'ECK', 'EMK',
       'ENT', 'ERD', 'ESR', 'FAS', 'FDM', 'FMG', 'FSH', 'GC', 'GRN',
       'GRS', 'HAS', 'HAT', 'HER', 'HKB', 'HKS', 'HOL', 'HRN', 'IMS',
       'INC', 'INP', 'ISR', 'JCI', 'JEN', 'JJK', 'JNG', 'JON', 'JRT',
       'KBM', 'KCX', 'KEL', 'KLK', 'KMV', 'KOS', 'KPI', 'KPK', 'KRA',
       'LAB', 'LAW', 'LEF', 'LMC', 'LOG', 'LYD', 'LYN', 'MAS', 'MB',
       'MCL', 'MGS', 'MGY', 'MIC', 'MKV', 'MMG', 'MOR', 'MPI', 'MSX',
       'MUZ', 'MvG', 'NET', 'NOL', 'NOR', 'OCT', 'OMN', 'OMY', 'PEQ',
       'PGH', 'PH', 'PIG', 'PIR', 'PKL', 'PMC', 'POM', 'PPR', 'PRR',
       'PTS', 'RAG', 'REI',

### RPI/NET

In [70]:
rpi_net_rankings = ordinals[(ordinals['SystemName'] == 'NET') | (ordinals['SystemName'] == 'RPI')]
rpi_net_rankings

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
3295,2003,37,RPI,1102,219
3296,2003,37,RPI,1103,253
3297,2003,37,RPI,1104,21
3298,2003,37,RPI,1105,300
3299,2003,37,RPI,1106,260
...,...,...,...,...,...
4593944,2022,128,NET,1468,199
4593945,2022,128,NET,1469,271
4593946,2022,128,NET,1470,214
4593947,2022,128,NET,1471,269


In [71]:
net_sub_q = '''
SELECT
    *,
    ROW_NUMBER() OVER(PARTITION BY Season, TeamID ORDER BY RankingDayNum DESC) AS RankingNum
FROM rpi_net_rankings
'''
net_sub = sqldf(net_sub_q)
net_sub.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank,RankingNum
0,2003,133,RPI,1102,158,1
1,2003,127,RPI,1102,159,2
2,2003,120,RPI,1102,156,3
3,2003,112,RPI,1102,162,4
4,2003,106,RPI,1102,149,5


In [72]:
q = '''
WITH net_window AS (
    SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY Season, TeamID ORDER BY RankingDayNum DESC) AS RankingNum
    FROM rpi_net_rankings
)

SELECT * 
FROM net_window a
INNER JOIN 
(SELECT Season, MIN(5, MAX(RankingNum)) rank_num
FROM net_window
GROUP BY 1) b
ON a.Season = b.Season
AND a.RankingNum = b.rank_num
'''
sqldf(q)

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank,RankingNum,Season,rank_num
0,2003,106,RPI,1102,149,5,2003,5
1,2003,106,RPI,1103,198,5,2003,5
2,2003,106,RPI,1104,27,5,2003,5
3,2003,106,RPI,1105,316,5,2003,5
4,2003,106,RPI,1106,236,5,2003,5
...,...,...,...,...,...,...,...,...
6879,2022,100,NET,1468,203,5,2022,5
6880,2022,100,NET,1469,281,5,2022,5
6881,2022,100,NET,1470,189,5,2022,5
6882,2022,100,NET,1471,279,5,2022,5


In [73]:
net_sub[(net_sub['Season'] == 2016) & (net_sub['TeamID'] == 1102)]

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank,RankingNum
51981,2016,133,RPI,1102,226,1
51982,2016,128,RPI,1102,221,2
51983,2016,121,RPI,1102,200,3


In [74]:
net_q = '''
WITH net_window AS (
    SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY Season, TeamID ORDER BY RankingDayNum DESC) AS RankingNum
    FROM rpi_net_rankings
)

SELECT 
    fp.Season, fp.TeamID,
    fp.final_net, lp.prev_net, 
    lp.prev_net - fp.final_net AS net_improvement
FROM
    (SELECT 
        Season, TeamID, OrdinalRank AS final_net
    FROM net_window
    WHERE RankingNum = 1) fp
LEFT JOIN 
    (SELECT 
        a.Season, a.TeamID, OrdinalRank AS prev_net 
    FROM net_window a
    INNER JOIN 
        (SELECT Season, MIN(5, MAX(RankingNum)) rank_num
        FROM net_window
        GROUP BY 1) b
    ON a.Season = b.Season
    AND a.RankingNum = b.rank_num) lp
ON fp.Season = lp.Season
AND fp.TeamID = lp.TeamID
'''
net_table = sqldf(net_q)
net_table.head()

,Season,TeamID,final_net,prev_net,net_improvement
0,2003,1102,158,149,-9
1,2003,1103,182,198,16
2,2003,1104,38,27,-11
3,2003,1105,313,316,3
4,2003,1106,248,236,-12


### Pomeroy

In [75]:
pom_rankings = ordinals[ordinals['SystemName'] == 'POM']

In [76]:
pom_sub_q = '''
SELECT
    *,
    ROW_NUMBER() OVER(PARTITION BY Season, TeamID ORDER BY RankingDayNum DESC) AS RankingNum
FROM pom_rankings
'''
pom_sub = sqldf(pom_sub_q)
pom_sub.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank,RankingNum
0,2003,133,POM,1102,160,1
1,2003,127,POM,1102,162,2
2,2003,120,POM,1102,165,3
3,2003,112,POM,1102,167,4
4,2003,106,POM,1102,151,5


In [77]:
pom_q = '''
WITH pom_window AS (
    SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY Season, TeamID ORDER BY RankingDayNum DESC) AS RankingNum
    FROM pom_rankings
)

SELECT 
    fp.Season, fp.TeamID,
    fp.final_pom, lp.prev_pom,
    lp.prev_pom - fp.final_pom AS pom_improvement
FROM
    (SELECT 
        Season, TeamID, OrdinalRank AS final_pom
    FROM pom_window
    WHERE RankingNum = 1) fp
LEFT JOIN 
    (SELECT 
        Season, TeamID, OrdinalRank AS prev_pom
    FROM pom_window
    WHERE RankingNum = 5) lp
ON fp.Season = lp.Season
AND fp.TeamID = lp.TeamID
'''
pom_table = sqldf(pom_q)
pom_table.head()

,Season,TeamID,final_pom,prev_pom,pom_improvement
0,2003,1102,160,151,-9
1,2003,1103,163,187,24
2,2003,1104,33,39,6
3,2003,1105,307,313,6
4,2003,1106,263,227,-36


In [78]:
rankings_df = net_table.merge(pom_table, on=['Season', 'TeamID'], how='outer')
rankings_df.head()

,Season,TeamID,final_net,prev_net,net_improvement,final_pom,prev_pom,pom_improvement
0,2003,1102,158.0,149.0,-9.0,160,151,-9
1,2003,1103,182.0,198.0,16.0,163,187,24
2,2003,1104,38.0,27.0,-11.0,33,39,6
3,2003,1105,313.0,316.0,3.0,307,313,6
4,2003,1106,248.0,236.0,-12.0,263,227,-36


## All together

In [79]:
team_season_preds_temp = team_season_stats_compact.merge(team_season_stats_vs_tourney_teams, on=['Season', 'TeamID'], how='left')
team_season_preds = team_season_preds_temp.merge(rankings_df, on=['Season', 'TeamID'], how='left')

In [80]:
team_season_preds

,Season,TeamID,total_games,num_wins,num_losses,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,...,away_wins_vs_tourney_teams,losses_vs_tourney_teams,losses_vs_non_tourney_teams,games_vs_tourney_teams,final_net,prev_net,net_improvement,final_pom,prev_pom,pom_improvement
0,2003,1102,28,12,16,0.428571,57.250000,57.000000,0.250000,15.878163,...,0.0,7.0,9.0,8.0,158.0,149.0,-9.0,160.0,151.0,-9.0
1,2003,1103,27,13,14,0.481481,78.777778,78.148148,0.629630,11.126039,...,1.0,1.0,13.0,2.0,182.0,198.0,16.0,163.0,187.0,24.0
2,2003,1104,28,17,11,0.607143,69.285714,65.000000,4.285714,13.149843,...,1.0,6.0,5.0,11.0,38.0,27.0,-11.0,33.0,39.0,6.0
3,2003,1105,26,7,19,0.269231,71.769231,76.653846,-4.884615,15.552212,...,0.0,3.0,16.0,4.0,313.0,316.0,3.0,307.0,313.0,6.0
4,2003,1106,28,13,15,0.464286,63.607143,63.750000,-0.142857,12.374266,...,0.0,4.0,11.0,5.0,248.0,236.0,-12.0,263.0,227.0,-36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6887,2015,1212,27,0,27,0.000000,51.111111,73.777778,-21.857143,16.513445,...,0.0,4.0,23.0,4.0,351.0,351.0,0.0,351.0,351.0,0.0
6888,2015,1363,28,0,28,0.000000,52.535714,70.678571,-17.517241,10.230518,...,0.0,5.0,23.0,5.0,336.0,337.0,1.0,349.0,348.0,-1.0
6889,2021,1152,9,0,9,0.000000,55.444444,88.666667,-29.900000,16.507271,...,0.0,4.0,5.0,4.0,346.0,346.0,0.0,346.0,356.0,10.0
6890,2022,1175,25,0,25,0.000000,60.600000,75.840000,-14.653846,12.569629,...,0.0,5.0,20.0,5.0,357.0,357.0,0.0,356.0,357.0,1.0


In [83]:
hf.team_lookup(1175, teams)

'Delaware St'

## Sanity checks

In [84]:
# number of teams per season
team_season_preds.groupby('Season').count()[['TeamID']]

,TeamID
Season,
2003,327
2004,326
2005,330
2006,334
2007,336
2008,342
2009,347
2010,347
2011,345


In [86]:
team_season_preds[team_season_preds.isnull().any(axis=1)]['Seed']

1764    None
2074    None
2193    None
2247    None
2254    None
3220    None
3723    None
3845    None
Name: Seed, dtype: object

All 8 teams with nulls were not tourney teams... going to drop them because we're not training on non-tourney teams instead of finding out why they have null values

In [87]:
team_season_preds.dropna(inplace=True)

In [91]:
team_season_preds.to_csv('data/generated_data/team_season_features.csv', index=False)

## Extra exploration

In [90]:
p21 = pom_rankings[pom_rankings['Season'] == 2020]

In [484]:
p21['RankingDayNum'].unique()

array([ 30,  37,  44,  51,  58,  65,  72,  79,  86,  93, 100, 107, 114,
       121, 128, 133])

In [494]:
for i in range(30,122,7):
    print(i)

30
37
44
51
58
65
72
79
86
93
100
107
114
121


In [535]:
p21[(p21['TeamID'] == 1124)]

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
4125221,2021,30,POM,1124,2
4132031,2021,37,POM,1124,2
4140983,2021,44,POM,1124,2
4150618,2021,51,POM,1124,1
4161866,2021,58,POM,1124,2
4173443,2021,65,POM,1124,2
4186144,2021,72,POM,1124,2
4199897,2021,79,POM,1124,2
4214360,2021,86,POM,1124,2
4229164,2021,93,POM,1124,2


In [505]:
p21[(p21['TeamID'] == team) & (p21['RankingDayNum'] == i)]['OrdinalRank'].iloc[0]

128

In [512]:
hf.team_lookup(1124, teams)

'Baylor'

In [541]:
pr = pom_rankings[pom_rankings['Season'] == 2019]
total_improvements = []
for team in pr['TeamID'].unique():
    is_improvement = []
    for i in range(30,115,7):
        is_improve = pr[(pr['TeamID'] == team) & (pr['RankingDayNum'] == i)]['OrdinalRank'].iloc[0] >= pr[(pr['TeamID'] == team) & (pr['RankingDayNum'] == i+7)]['OrdinalRank'].iloc[0]
        is_improvement.append(is_improve)
    print(team, is_improvement, sum(is_improvement))
    total_improvements.append((hf.team_lookup(team, teams), team, sum(is_improvement)))

1101 [True, False, True, False, False, True, True, True, True, False, True, False, True] 8
1102 [False, True, False, True, False, True, True, True, False, False, True, True, True] 8
1103 [False, True, True, True, True, False, False, True, True, True, True, True, False] 9
1104 [False, False, True, True, True, False, True, True, True, True, False, False, True] 8
1105 [True, True, True, False, True, True, False, True, True, True, False, True, True] 10
1106 [False, False, False, False, True, False, True, True, False, False, True, True, False] 5
1107 [True, False, False, True, False, False, False, True, True, False, False, True, False] 5
1108 [True, True, False, True, False, True, False, True, True, True, False, True, False] 8
1110 [True, False, False, False, True, True, False, True, True, False, False, True, True] 7
1111 [False, False, False, False, True, False, False, True, True, True, True, False, True] 6
1112 [False, False, False, True, False, True, False, False, True, False, False, Tru

1202 [True, True, False, False, True, True, True, False, True, True, True, False, True] 9
1203 [False, False, False, True, True, False, False, True, True, False, False, True, False] 5
1204 [True, True, False, True, True, False, True, True, False, True, True, True, False] 9
1205 [True, False, True, True, False, True, False, False, False, True, True, False, True] 7
1206 [True, True, True, True, True, True, False, True, True, False, False, False, True] 9
1207 [True, False, False, True, True, False, False, True, True, True, False, True, True] 8
1208 [False, False, True, True, False, True, False, True, False, False, False, True, True] 6
1209 [True, False, True, True, True, False, False, False, True, False, True, False, True] 7
1210 [True, False, False, True, True, True, False, True, False, False, False, True, False] 6
1211 [False, False, True, True, True, False, True, True, True, True, True, True, True] 10
1212 [False, True, False, False, False, False, True, False, True, True, True, False, 

1300 [True, False, True, True, True, True, False, False, False, False, True, True, True] 8
1301 [True, True, True, False, True, False, False, True, False, True, True, True, True] 9
1303 [False, True, True, False, True, True, True, True, True, False, True, True, False] 9
1304 [True, True, True, False, False, True, True, False, False, False, True, False, False] 6
1305 [True, False, False, True, False, True, False, True, True, True, True, False, False] 7
1306 [True, False, False, True, True, True, False, False, False, True, True, False, True] 7
1307 [False, True, False, False, True, False, True, False, True, True, True, True, False] 7
1308 [True, False, True, True, False, False, True, True, True, True, False, False, True] 8
1309 [True, True, False, False, True, True, False, True, True, False, True, True, False] 8
1310 [True, True, False, False, False, False, True, True, False, False, False, False, True] 5
1311 [True, False, False, True, False, True, False, False, False, True, False, True,

1392 [True, False, False, True, True, False, True, False, False, True, False, False, True] 6
1393 [True, False, False, True, False, False, True, False, True, False, False, True, True] 6
1394 [True, False, True, True, True, True, True, True, True, False, False, True, False] 9
1395 [True, True, True, True, False, True, True, False, False, True, False, False, False] 7
1396 [True, False, True, False, True, True, False, True, False, True, True, True, False] 8
1397 [True, True, False, True, True, True, True, False, True, True, False, False, True] 9
1398 [True, False, False, True, False, False, True, False, True, True, False, True, False] 6
1399 [False, True, False, False, True, True, False, True, True, False, False, False, True] 6
1400 [False, True, False, True, True, False, True, False, True, True, False, True, True] 8
1401 [False, True, True, False, True, True, False, True, False, True, True, True, True] 9
1402 [True, True, True, True, True, True, True, True, False, False, True, False, Fal

In [542]:
total_improvements.sort(key=lambda x: x[2], reverse=True)

In [543]:
total_improvements

[('Delaware St', 1175, 12),
 ('FL Atlantic', 1194, 12),
 ('Michigan St', 1277, 12),
 ('Virginia', 1438, 12),
 ('Charleston So', 1149, 11),
 ('CS Northridge', 1169, 11),
 ('Duke', 1181, 11),
 ('Houston', 1222, 11),
 ('LSU', 1261, 11),
 ('Memphis', 1272, 11),
 ('Norfolk St', 1313, 11),
 ('Northwestern LA', 1322, 11),
 ('Wofford', 1459, 11),
 ('Alabama A&M', 1105, 10),
 ('Austin Peay', 1122, 10),
 ('Bowling Green', 1132, 10),
 ('Buffalo', 1138, 10),
 ('Chattanooga', 1151, 10),
 ('Gonzaga', 1211, 10),
 ('High Point', 1219, 10),
 ('Jacksonville St', 1240, 10),
 ('Kentucky', 1246, 10),
 ('Maryland', 1268, 10),
 ('Mississippi', 1279, 10),
 ('Montana St', 1286, 10),
 ('Murray St', 1293, 10),
 ('Navy', 1298, 10),
 ('North Carolina', 1314, 10),
 ('Sam Houston St', 1358, 10),
 ('Savannah St', 1366, 10),
 ('South Florida', 1378, 10),
 ('Stetson', 1391, 10),
 ('Utah St', 1429, 10),
 ('Utah Valley', 1430, 10),
 ('Virginia Tech', 1439, 10),
 ('Akron', 1103, 9),
 ('Belmont', 1125, 9),
 ('Canisius', 11

In [551]:
pr[(pr['TeamID'] == 1274)]

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
3496394,2019,9,POM,1274,18
3502823,2019,16,POM,1274,21
3511723,2019,23,POM,1274,23
3522035,2019,30,POM,1274,35
3534171,2019,37,POM,1274,45
3548421,2019,44,POM,1274,46
3563382,2019,51,POM,1274,49
3578343,2019,58,POM,1274,56
3594713,2019,65,POM,1274,67
3612145,2019,72,POM,1274,68
